In [12]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [13]:
import pyomo.environ as pe
import pyomo.opt as po

In [14]:
# Initialize Model
model = pe.ConcreteModel()

# Sets
model.F = pe.Set(initialize = factories ,ordered = False)
model.T = pe.Set(initialize = transshipment_points ,ordered = False)
model.C = pe.Set(initialize = customers ,ordered = False)
model.M = pe.Set(initialize = modes ,ordered = False)
model.P = pe.Set(initialize = periods, ordered = False)

# Decision Variables
model.x = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeReals)                   # direct tonnes                         !!!TODO : meer uitleg !!!
model.y = pe.Var(model.T, model.C , model.P, domain = pe.NonNegativeReals)                  # indirect tonnes                       !!!TODO : meer uitleg !!!
model.q_bl = pe.Var(model.F, block_trans, model.P, domain = pe.NonNegativeReals)            # quantity block mode                   !!!TODO : meer uitleg !!!  
model.q_s_w = pe.Var(model.F, single_trans, model.P, domain = pe.NonNegativeReals)          # quantity single wagon mode            !!!TODO : meer uitleg !!!
model.q_ss = pe.Var(model.F, shortsea_trans, model.P, domain = pe.NonNegativeReals)         # quantity shortsea ship mode           !!!TODO : meer uitleg !!!
model.q_ba = pe.Var(model.F, barge_trans, model.P, domain = pe.NonNegativeReals)            # quantity barge mode                   !!!TODO : meer uitleg !!!
model.td = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeIntegers)               # number of trucks via direct route     !!!TODO : meer uitleg !!!
model.tid = pe.Var(model.T, model.C, model.P, domain = pe.NonNegativeIntegers)              # number of trucks via indirect route   !!!TODO : meer uitleg !!!

In [15]:
objExpr = sum(dist_fact_cust[i, j] * transfer_cost * model.x[i,j,p] for i in model.F for j in model.C for p in model.P)     \
        + sum(block_train_cost[i, j] * model.q_bl[i,j,p] for i in model.F for j in block_trans for p in model.P)            \
        + sum(single_wagon_cost[i, j] * model.q_s_w[i,j,p] for i in model.F for j in single_trans for p in model.P)         \
        + sum(shortsea_cost[i, j] * model.q_ss[i,j,p] for i in model.F for j in shortsea_trans for p in model.P)            \
        + sum(barge_cost[i, j] * model.q_ba[i,j,p] for i in model.F for j in barge_trans for p in model.P)                  \
        + sum(model.td[i,j,p] * truck_fixed_cost for i in model.F for j in model.C for p in model.P)                        \
        + sum(model.tid[i,j,p] * truck_fixed_cost for i in model.T for j in model.C for p in model.P)

model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [16]:
model.trucksDirect = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.C:
            expression = model.td[i,j,p] >= model.x[i,j,p] / truck_cap
            model.trucksDirect.add(expression)

model.trucksIndirect = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.C:
            expression = model.tid[i,j,p] >= model.y[i,j,p] / truck_cap
            model.trucksIndirect.add(expression)

model.prodCap = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        expression = sum(model.x[i,j,p] for j in model.C) + sum(model.q_bl[i,j,p] for j in block_trans) + sum(model.q_s_w[i,j,p] for j in single_trans) + sum(model.q_ss[i,j,p] for j in shortsea_trans) + sum(model.q_ba[i,j,p] for j in barge_trans) <= production_cap[i]
        model.prodCap.add(expression)

model.meetDemand = pe.ConstraintList()
for p in model.P:
    for j in model.C:
        expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) >= demands[p][j]
        model.meetDemand.add(expression)

model.flowBalance = pe.ConstraintList()
for p in model.P:
    for j in block_trans:
        expression_bl = sum(model.q_bl[i,j,p] for i in model.F) 
        expression_s_w = sum(model.q_s_w[i,j,p] for i in model.F)
    for j in shortsea_trans:
        expression_ss = sum(model.q_ss[i,j,p] for i in model.F) 
    for j in barge_trans:
        expression_ba = sum(model.q_ba[i,j,p] for i in model.F) 
    for j in model.T: 
        expression = expression_bl + expression_s_w + expression_ss + expression_ba == sum(model.y[j,k,p] for k in model.C)
        model.flowBalance.add(expression)

In [17]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-28
Read LP format model from file /var/folders/nh/v22djfjs2712hmlms5xl3z8h0000gs/T/tmprfpkf6vu.pyomo.lp
Reading time = 0.01 seconds
x1871: 981 rows, 1871 columns, 4311 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 981 rows, 1871 columns and 4311 nonzeros
Model fingerprint: 0x6bb06f93
Variable types: 1071 continuous, 800 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [2e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 4127715.2500
Presolve removed 1 rows and 241 columns
Presolve time: 0.12s
Presolved: 980 rows, 1630 columns, 3440 nonzeros
Variable types: 830 continuous, 800 integer (30 binary)

Root relaxation: objective 3.462003e+04, 896 iterations, 0.00 seconds

In [21]:
print("Objective value = "+ str(pe.value(model.obj)))
print("Direct Tonnes:\n")
for k in model.P:
    for i in model.F:
        for j in model.C:
            if(pe.value(model.x[i,j,k]) > 0):
                print(str(model.x[i,j,k]) + " = " + str(pe.value(model.x[i,j,k])))

print("\Indirect Tonnes:\n")
for k in model.P:
    for i in model.T:
        for j in model.C:
            if(pe.value(model.y[i,j,k]) > 0):
                print(str(model.y[i,j,k]) + " = " + str(pe.value(model.y[i,j,k])))

Objective value = 37742.76750000009
Direct Tonnes:

